In [1]:
with open('input.txt', "r") as file:
    data = file.read()


print(data.splitlines()[0])

mirrored gold bags contain 3 light teal bags.


In [2]:
test1 = '''light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags.'''

In [3]:
test2 = '''shiny gold bags contain 2 dark red bags.
dark red bags contain 2 dark orange bags.
dark orange bags contain 2 dark yellow bags.
dark yellow bags contain 2 dark green bags.
dark green bags contain 2 dark blue bags.
dark blue bags contain 2 dark violet bags.
dark violet bags contain no other bags.'''

In [4]:
def make_rules(data):
    """
    parse puzzle into dict
    
    f(str) -> {
        key_col_1: [(x_1_1 col_1_1), (x_1_2 col_1_2), ...],
        key_col_2: [(x_2_1 col_2_1), (x_2_2 col_2_2), ...],
        ...
    }
    """
    rules = {}
    for line in data.splitlines():
        line = line.replace(' bags', '')
        line = line.replace(' bag', '')
        line = line.replace('.', '')
        line = line.replace('no other', '0 any')
        container, contents = line.split(' contain ')
        contents = contents.split(', ')
        rules[container] = contents
    return rules

rules = make_rules(data)


# Part 1

In [5]:
def can_contain_color_x(color, rules, color_x):
    """0 for no, number of colors for yes"""
    res = 0
    if color == color_x: # check for cycled containing
        if color_x in [neighb.split(' ', 1)[1] for neighb in rules[color_x]]:
            res = 1
        else:
            res = max([
                can_contain_color_x(neighb.split(' ', 1)[1], rules, color_x)
                for neighb in rules[color]
                if int(neighb[0]) > 0
            ])
    else:
        visited_colors = set()
        x_to_visit = []
        to_visit = set([color])

        while to_visit:
            new_to_visit = set()
            for cur_col in to_visit:
                visited_colors.add(cur_col)
                cur_neighbs = [neighb.split(' ', 1)[1] for neighb in rules[cur_col]
                               if int(neighb[0]) > 0
                               and neighb not in visited_colors]
                new_to_visit.update(cur_neighbs)
            to_visit = new_to_visit
        if color_x in visited_colors:
            res = len(visited_colors)

    return res

def count_valid_outermosts(color_x, rules):
    n_valid = 0
    for color in rules:
        if can_contain_color_x(color, rules, color_x):
            n_valid += 1
    return n_valid


assert(count_valid_outermosts('shiny gold', make_rules(test1)) == 4)

print(count_valid_outermosts('shiny gold', make_rules(data)))

278


# Part 2

In [6]:
def count_bags(color, n, rules):
    """assuming no cycles, otherwise result is +infinity"""
    res = 0
    for neighb in rules[color]:
        x, new_color = neighb.split(' ', 1)
        x = int(x)
        if x:
            res += count_bags(new_color, x*n, rules)
    return res + n


assert(count_bags('shiny gold', 1, make_rules(test1)) - 1 == 32)
assert(count_bags('shiny gold', 1, make_rules(test2)) - 1 == 126)

print(count_bags('shiny gold', 1, make_rules(data)) - 1)

45157
